<a href="https://colab.research.google.com/github/Vishnupriya-Selvraj/Agentic_AI_Workshop/blob/main/Healthcare_Policy_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq -q
!pip install python-dotenv -q
!pip install gradio -q

print("✅ Libraries installed successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.7 MB/s eta 0:00:00
✅ Libraries installed successfully!


In [ ]:
import os
from groq import Groq
import json
from typing import Dict, List, Any
import re
from datetime import datetime
import gradio as gr


In [ ]:
GROQ_API_KEY = "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"

In [ ]:
try:
    client = Groq(api_key=GROQ_API_KEY)
    print("✅ Groq client initialized successfully!")
except Exception as e:
    print(f"❌ Error initializing Groq client: {e}")
    print("Please check your API key!")

# Step 4: Define HealthSecure Insurance Policy Database
POLICY_DATABASE = {
    "Basic Health Plan": {
        "description": "Most affordable option for essential healthcare needs",
        "coverage": [
            "In-Patient Hospitalization (shared room)",
            "Emergency Room Services",
            "Basic Diagnostic Services during hospitalization",
            "Emergency Prescriptions (up to $250 annually)"
        ],
        "eligibility": "Individuals aged 18-55",
        "premium": "$150 per month",
        "deductible": "$1,500 per year",
        "coverage_limit": "$100,000",
        "network_hospitals": "2,500+",
        "waiting_periods": {"pre_existing": "12 months"},
        "exclusions": ["Outpatient services", "Wellness benefits", "Maternity", "Dental care"]
    },

    "Family Health Plus Plan": {
        "description": "Comprehensive coverage for families including in-patient and out-patient services",
        "coverage": [
            "Comprehensive In-Patient Hospitalization",
            "Family Doctor Visits (10 per year per member)",
            "Specialist Consultations (15 annually)",
            "Diagnostics and Lab Tests",
            "Emergency Care and Transport",
            "Prescription Drugs (up to $500 per member)"
        ],
        "eligibility": "Families with dependent children up to age 25",
        "premium": "$350 per month",
        "deductible": "$1,000 individual / $3,000 family",
        "coverage_limit": "$500,000",
        "network_hospitals": "5,000+",
        "waiting_periods": {"pre_existing": "6 months", "maternity": "9 months"},
        "discounts": ["10% family bundle discount", "5% preventive health discount"]
    },

    "Comprehensive Health & Wellness Plan": {
        "description": "Holistic health support with wellness resources",
        "coverage": [
            "All Family Health Plus benefits",
            "Preventive Care Services",
            "Mental Health Support (6 sessions per year)",
            "Wellness Programs and fitness coaching",
            "Additional Specialist Coverage",
            "Extended Prescription Coverage (up to $1,000 per member)"
        ],
        "eligibility": "Individuals and families with no age restrictions",
        "premium": "$500 per month",
        "deductible": "$750 individual / $2,000 family",
        "coverage_limit": "$1,000,000",
        "network_hospitals": "7,500+",
        "waiting_periods": {"pre_existing": "6 months", "mental_health": "3 months"},
        "discounts": ["10% no-claim bonus up to 30%", "5% annual payment discount"]
    },

    "Senior Health Security Plan": {
        "description": "Specialized coverage for seniors with extended healthcare",
        "coverage": [
            "Comprehensive Hospitalization with private rooms",
            "Unlimited Specialized Consultations",
            "Routine Prescriptions (up to $2,000 annually)",
            "Vision and Dental Benefits",
            "Senior Wellness Programs"
        ],
        "eligibility": "Individuals aged 55 and above",
        "premium": "$600 per month",
        "deductible": "$1,000 annually",
        "coverage_limit": "$750,000",
        "network_hospitals": "5,000+ with senior-focused facilities",
        "waiting_periods": {"pre_existing": "3 months", "vision_dental": "No waiting period"},
        "special_features": ["Priority claims processing", "Dedicated senior support"]
    }
}

# Add-on Coverage Options
ADD_ON_OPTIONS = {
    "Dental & Vision Add-On": {
        "cost": "$50 per month per member",
        "coverage": ["Two dental exams per year", "Annual eye exam", "$200 limit for corrective lenses"]
    },
    "Maternity & Newborn Care Add-On": {
        "cost": "$75 per month",
        "coverage": ["Pre-natal and post-natal care", "Delivery expenses", "Newborn immunizations"],
        "waiting_period": "9 months"
    },
    "International Travel Medical Insurance": {
        "cost": "$40 per month",
        "coverage": ["Emergency medical coverage abroad up to $100,000", "24/7 global helpline"]
    }
}

print("✅ Policy database loaded successfully!")

✅ Groq client initialized successfully!
✅ Policy database loaded successfully!


In [ ]:
class HealthcarePolicyAgent:
    def __init__(self, client):
        self.client = client
        self.conversation_history = []
        self.user_profile = {}

    def create_system_prompt(self):
        policy_info = ""
        for plan_name, details in POLICY_DATABASE.items():
            policy_info += f"\n{plan_name}:\n"
            policy_info += f"- Description: {details['description']}\n"
            policy_info += f"- Premium: {details['premium']}\n"
            policy_info += f"- Eligibility: {details['eligibility']}\n"
            policy_info += f"- Coverage Limit: {details['coverage_limit']}\n"

        return f"""You are a Healthcare Policy Sales Agent for HealthSecure Insurance Ltd.
        Your role is to recommend the most suitable insurance policy based on user details.

        Available Plans:{policy_info}

        Add-on Options:
        - Dental & Vision Add-On: $50/month per member
        - Maternity & Newborn Care Add-On: $75/month (for families)
        - International Travel Medical Insurance: $40/month

        Guidelines:
        - Ask clarifying questions about age, family status, dependents, and special needs
        - Recommend policies based on user's specific requirements
        - Explain coverage details clearly and concisely
        - Mention relevant add-ons when appropriate
        - Stay focused on healthcare policy recommendations
        - Be helpful, professional, and friendly
        - Always provide premium costs and key benefits

        Structure your recommendations:
        1. Recommended plan(s) and brief reason why
        2. Key coverage highlights (3-4 main benefits)
        3. Premium and deductible information
        4. Relevant add-ons if applicable
        5. Ask if they need clarification on anything
        """

    def extract_user_info(self, query: str):
        """Extract key information from user query"""
        query_lower = query.lower()

        # Extract age
        age_patterns = [
            r'\b(\d{1,2})\s*(?:years?\s*old|yr|y\.o\.)',
            r'age\s*(\d{1,2})',
            r"i'm\s*(\d{1,2})"
        ]

        for pattern in age_patterns:
            match = re.search(pattern, query_lower)
            if match:
                age = int(match.group(1))
                if 10 <= age <= 100:  # Reasonable age range
                    self.user_profile['age'] = age
                    break

        # Family status
        if any(word in query_lower for word in ['family', 'married', 'spouse', 'wife', 'husband']):
            self.user_profile['coverage_type'] = 'family'
        elif any(word in query_lower for word in ['single', 'individual', 'just me']):
            self.user_profile['coverage_type'] = 'individual'

        # Special needs
        special_needs = []
        if any(word in query_lower for word in ['dental', 'teeth', 'tooth']):
            special_needs.append('dental')
        if any(word in query_lower for word in ['vision', 'eye', 'glasses', 'contacts']):
            special_needs.append('vision')
        if any(word in query_lower for word in ['maternity', 'pregnancy', 'baby', 'newborn']):
            special_needs.append('maternity')
        if any(word in query_lower for word in ['travel', 'international']):
            special_needs.append('travel')
        if any(word in query_lower for word in ['mental', 'therapy', 'counseling']):
            special_needs.append('mental_health')
        if any(word in query_lower for word in ['wellness', 'fitness', 'preventive']):
            special_needs.append('wellness')

        if special_needs:
            self.user_profile['special_needs'] = special_needs

    def get_policy_recommendation(self, user_query: str) -> str:
        try:
            # Extract user information
            self.extract_user_info(user_query)

            # Create enhanced system prompt
            system_prompt = self.create_system_prompt()
            if self.user_profile:
                system_prompt += f"\n\nUser Profile Information: {self.user_profile}"

            # Create messages for the conversation
            messages = [{"role": "system", "content": system_prompt}]

            # Add conversation history
            for msg in self.conversation_history[-6:]:  # Keep last 6 messages
                messages.append(msg)

            # Add current user query
            messages.append({"role": "user", "content": user_query})

            # Get response from Groq
            response = self.client.chat.completions.create(
                model="llama3-8b-8192",  # Free model on Groq
                messages=messages,
                temperature=0.7,
                max_tokens=1000,
                top_p=0.9
            )

            # Extract the response
            agent_response = response.choices[0].message.content

            # Update conversation history
            self.conversation_history.append({"role": "user", "content": user_query})
            self.conversation_history.append({"role": "assistant", "content": agent_response})

            return agent_response

        except Exception as e:
            return f"I apologize for the technical difficulty. Please try rephrasing your question. Error details: {str(e)}"

    def reset_conversation(self):
        """Reset the conversation history and user profile"""
        self.conversation_history = []
        self.user_profile = {}

    def get_conversation_summary(self):
        """Get a summary of the conversation"""
        return {
            "total_messages": len(self.conversation_history),
            "user_profile": self.user_profile,
            "last_interaction": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

print("✅ Healthcare Policy Agent class created successfully!")

✅ Healthcare Policy Agent class created successfully!


In [ ]:
agent = HealthcarePolicyAgent(client)

In [ ]:
def chat_response(message, history):
    """Function to handle chat responses for Gradio interface"""
    if not message.strip():
        return "", history

    try:
        # Get response from agent
        response = agent.get_policy_recommendation(message)

        # Update history
        history.append([message, response])

        return "", history
    except Exception as e:
        error_msg = f"Sorry, I encountered an error: {str(e)}"
        history.append([message, error_msg])
        return "", history

def clear_conversation():
    """Function to clear conversation history"""
    agent.reset_conversation()
    return []

def get_user_profile():
    """Function to display current user profile"""
    profile = agent.user_profile
    if not profile:
        return "No user profile information collected yet."

    profile_text = "**Current User Profile:**\n"
    for key, value in profile.items():
        profile_text += f"- {key.replace('_', ' ').title()}: {value}\n"

    return profile_text

def show_available_plans():
    """Function to display available insurance plans"""
    plans_text = "## Available Insurance Plans\n\n"

    for plan_name, details in POLICY_DATABASE.items():
        plans_text += f"### {plan_name}\n"
        plans_text += f"**Premium:** {details['premium']}\n"
        plans_text += f"**Description:** {details['description']}\n"
        plans_text += f"**Eligibility:** {details['eligibility']}\n"
        plans_text += f"**Coverage Limit:** {details['coverage_limit']}\n"
        plans_text += f"**Network Hospitals:** {details['network_hospitals']}\n\n"

    plans_text += "## Add-on Options\n\n"
    for addon_name, details in ADD_ON_OPTIONS.items():
        plans_text += f"### {addon_name}\n"
        plans_text += f"**Cost:** {details['cost']}\n"
        plans_text += f"**Coverage:** {', '.join(details['coverage'])}\n\n"

    return plans_text


In [23]:
def create_gradio_interface():
    """Create and return the Gradio interface"""

    with gr.Blocks(
        title="HealthSecure Insurance Policy Assistant",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            max-width: 1200px !important;
        }
        .chat-container {
            height: 500px;
        }
        """
    ) as interface:

        # Header
        gr.Markdown("""
        # 🏥 HealthSecure Insurance Policy Assistant

        Welcome to HealthSecure Insurance! I'm your personal policy advisor.
        Tell me about your healthcare needs, and I'll recommend the best insurance plan for you.

        **Example questions you can ask:**
        - "I'm 30 years old and looking for basic coverage"
        - "We're a family of 4 and need comprehensive coverage"
        - "I'm 60 and need senior-focused healthcare"
        - "What plans include dental and vision?"
        """)

        with gr.Row():
            # Main chat interface
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    label="Chat with Your Policy Advisor",
                    height=500,
                    show_label=True,
                    container=True,
                    bubble_full_width=False
                )

                with gr.Row():
                    msg = gr.Textbox(
                        label="Your Message",
                        placeholder="Ask me about insurance plans, coverage, or pricing...",
                        lines=2,
                        max_lines=5,
                        scale=4
                    )
                    send_btn = gr.Button("Send", variant="primary", scale=1)

                with gr.Row():
                    clear_btn = gr.Button("Clear Conversation", variant="secondary")
                    profile_btn = gr.Button("Show My Profile", variant="secondary")

            # Side panel
            with gr.Column(scale=1):
                gr.Markdown("### Quick Actions")

                plans_btn = gr.Button("View All Plans", variant="outline")

                # Sample questions
                gr.Markdown("### Sample Questions")
                sample_questions = [
                    "I'm 28, single, need basic coverage",
                    "Family of 4, want comprehensive plan",
                    "Senior, 65, need specialized care",
                    "Young couple planning pregnancy"
                ]

                for question in sample_questions:
                    sample_btn = gr.Button(question, variant="outline", size="sm")
                    sample_btn.click(
                        lambda q=question: (q, []),
                        outputs=[msg, chatbot]
                    )

        # Output areas
        profile_output = gr.Markdown(label="User Profile", visible=False)
        plans_output = gr.Markdown(label="Available Plans", visible=False)

        # Event handlers
        def send_message(message, history):
            return chat_response(message, history)

        def show_profile():
            return gr.update(value=get_user_profile(), visible=True)

        def show_plans():
            return gr.update(value=show_available_plans(), visible=True)

        # Wire up the events
        send_btn.click(
            send_message,
            inputs=[msg, chatbot],
            outputs=[msg, chatbot]
        )

        msg.submit(
            send_message,
            inputs=[msg, chatbot],
            outputs=[msg, chatbot]
        )

        clear_btn.click(
            clear_conversation,
            outputs=[chatbot]
        )

        profile_btn.click(
            show_profile,
            outputs=[profile_output]
        )

        plans_btn.click(
            show_plans,
            outputs=[plans_output]
        )

    return interface


In [24]:
def test_agent_with_samples():
    """Test the agent with sample queries and display results"""
    sample_queries = [
        "I'm 28 years old, single, and looking for basic coverage for emergencies only. What do you recommend?",
        "We're a family of 4 (parents aged 32 and 30, kids aged 5 and 8). We want comprehensive coverage including dental and vision.",
        "I'm 62 years old and need coverage for routine medications and specialized care. What's best for seniors?",
        "Young married couple, both 26, planning to have a baby next year. What coverage should we get?"
    ]

    print("🧪 TESTING HEALTHCARE POLICY AGENT")
    print("=" * 80)

    for i, query in enumerate(sample_queries, 1):
        print(f"\n📋 Test Case {i}:")
        print(f"Query: {query}")
        print("\n🤖 Agent Response:")
        print("-" * 40)

        response = agent.get_policy_recommendation(query)
        print(response)
        print("-" * 40)

        # Show extracted user profile
        if agent.user_profile:
            print(f"\n📊 Extracted Profile: {agent.user_profile}")

        print("\n" + "=" * 80)

        # Reset for next test
        agent.reset_conversation()


In [ ]:
if __name__ == "__main__":
    print("✅ Creating Gradio interface...")

    # Create the interface
    interface = create_gradio_interface()

    print("✅ Gradio interface created successfully!")
    print("🚀 Launching the Healthcare Policy Assistant...")

    # Launch the interface
    interface.launch(
        share=True,  # Creates a public link
        debug=True,  # Enable debug mode
        show_error=True,  # Show errors in the interface
        server_port=7860,  # Default Gradio port
        server_name="0.0.0.0"  # Allow external access
    )

    print("✅ Application launched successfully!")
    print("📱 You can now interact with the Healthcare Policy Assistant through the web interface!")

# Optional: Run the original test function
print("\n" + "="*50)
print("OPTIONAL: Run test_agent_with_samples() to see console output")
print("="*50)

✅ Creating Gradio interface...
✅ Gradio interface created successfully!
🚀 Launching the Healthcare Policy Assistant...


<ipython-input-23-1477780242>:34: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
<ipython-input-23-1477780242>:34: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db7f47bb20f2c7e397.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
